**First**

In [3]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

     |████████████████████████████████| 102kB 1.8MB/s ta 0:00:01     |██████████                      | 30kB 9.9MB/s eta 0:00:01
Packages installed.


In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 6.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html5lib')

In [31]:
postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [32]:
toronto_data.shape

(103, 3)

**Second**

In [17]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [20]:
geo_df=pd.read_csv('Geospatial_Coordinates.csv')

In [21]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
Final_df = pd.concat([toronto_data, geo_df[['Latitude', 'Longitude']]], axis=1)
display(Final_df)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437


**Third**

In [37]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print ('finish')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [38]:
Downtown_df = Final_df[Final_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
display(Downtown_df)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
4,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
5,M6G,Downtown Toronto,Christie,43.753259,-79.329656
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.689574,-79.383160


In [39]:
# get location to create map of Toronto
address = 'Downtown Toronto, Toronto'
geolocator = Nominatim(user_agent="dt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [40]:

# create map of Downtown Toronto with Neighbourhoods 
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Downtown_df['Latitude'], Downtown_df['Longitude'], Downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [41]:
# entering my codes for Foursquare 
CLIENT_ID = 'QOGRXTV3MULPJQWU4BSX2QJVGICU4MY2BBVQAKOULYY5CUMR' # your Foursquare ID
CLIENT_SECRET = 'KCDGNLTMXIYJYNDMZ2SENA21QYD2E1HJUOQEWB0IRPLGGGH1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QOGRXTV3MULPJQWU4BSX2QJVGICU4MY2BBVQAKOULYY5CUMR
CLIENT_SECRET:KCDGNLTMXIYJYNDMZ2SENA21QYD2E1HJUOQEWB0IRPLGGGH1


In [42]:
Downtown_df=Downtown_df.drop('PostalCode', 1)
Downtown_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,Downtown Toronto,St. James Town,43.799525,-79.318389
3,Downtown Toronto,Berczy Park,43.757490,-79.374714
4,Downtown Toronto,Central Bay Street,43.782736,-79.442259


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
Downtown_venues = getNearbyVenues(names=Downtown_df['Neighborhood'],
                                   latitudes=Downtown_df['Latitude'],
                                   longitudes=Downtown_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [45]:
print(Downtown_venues.shape)
Downtown_venues.head()

(123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [46]:
# check why Berczy Park dissaparead from the list  check=Downtown_venues[Downtown_venues['Neighborhood'] == 'Regent Park, Harbourfront'].reset_index(drop=True)

neighborhood_latitude = Downtown_df.loc[3, 'Latitude'] # Berczy Park latitude value
neighborhood_longitude = Downtown_df.loc[3, 'Longitude'] # Berczy Park longitude value

neighborhood_name = Downtown_df.loc[3, 'Neighborhood'] # Berczy Park name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results
# does not return locattions  - this exmplains reduced frame by one neighbbourhood in further analysis

Latitude and longitude values of Berczy Park are 43.7574902, -79.37471409999999.


{'meta': {'code': 200, 'requestId': '607348aed9499c362ac0e193'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'St. Andrew - Windfields',
  'headerFullLocation': 'St. Andrew - Windfields, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.7619902045, 'lng': -79.3684954001132},
   'sw': {'lat': 43.7529901955, 'lng': -79.38093279988678}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [47]:
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,4,4,4,4,4,4
Christie,3,3,3,3,3,3
Church and Wellesley,8,8,8,8,8,8
"Commerce Court, Victoria Hotel",1,1,1,1,1,1
"First Canadian Place, Underground city",1,1,1,1,1,1
"Garden District, Ryerson",5,5,5,5,5,5
"Harbourfront East, Union Station, Toronto Islands",4,4,4,4,4,4
"Kensington Market, Chinatown, Grange Park",35,35,35,35,35,35


In [48]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 68 uniques categories.


In [49]:
# analizing each neighbourhood 
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot.head()

,Neighborhood,Airport,Auto Workshop,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,...,Skate Park,Skating Rink,Smoothie Shop,Spa,Steakhouse,Supermarket,Sushi Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Regent Park, Harbourfront",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#grouping by neighbourhood to analyze frequencies
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Airport,Auto Workshop,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,...,Skate Park,Skating Rink,Smoothie Shop,Spa,Steakhouse,Supermarket,Sushi Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.071429,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.071429,...,0.071429,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000
2,Christie,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Church and Wellesley,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,"Commerce Court, Victoria Hotel",0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,"First Canadian Place, Underground city",0.0,0.000000,0.000000,0.000000,1.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,"Garden District, Ryerson",0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.20,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
7,"Harbourfront East, Union Station, Toronto Islands",0.0,0.000000,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,...,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8,"Kensington Market, Chinatown, Grange Park",0.0,0.000000,0.028571,0.028571,0.0,0.00,0.028571,0.000000,0.000000,...,0.000000,0.00,0.028571,0.028571,0.000000,0.000000,0.057143,0.00,0.028571,0.028571
9,"Regent Park, Harbourfront",0.0,0.000000,0.111111,0.000000,0.0,0.00,0.000000,0.111111,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000


In [51]:
# determning most common venues  
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
# displaying top 10 most common venues by neighnourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a dataframe with top 10 venues for each neighbourhoud 
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

display(neighborhoods_venues_sorted)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Garden,Skate Park,Comic Shop,Park,Light Rail Station,Pizza Place,Butcher,Burrito Place,Brewery,Restaurant
1,Central Bay Street,Pizza Place,Supermarket,Coffee Shop,Pharmacy,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,College Stadium
2,Christie,Park,Pool,Food & Drink Shop,Yoga Studio,Electronics Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant
3,Church and Wellesley,Pizza Place,Coffee Shop,Intersection,Playground,Chinese Restaurant,Discount Store,Sandwich Place,Farm,Farmers Market,Falafel Restaurant
4,"Commerce Court, Victoria Hotel",Park,Yoga Studio,Falafel Restaurant,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farm,Garden Center
5,"First Canadian Place, Underground city",Baseball Field,Yoga Studio,Farmers Market,Diner,Discount Store,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant
6,"Garden District, Ryerson",General Entertainment,Skating Rink,Farm,Café,College Stadium,Baseball Field,Dessert Shop,Furniture / Home Store,French Restaurant,Food & Drink Shop
7,"Harbourfront East, Union Station, Toronto Islands",Park,Beer Store,Skating Rink,Curling Ice,Falafel Restaurant,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store
8,"Kensington Market, Chinatown, Grange Park",Café,Pub,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,French Restaurant,Diner,Vegetarian / Vegan Restaurant,Gourmet Shop
9,"Regent Park, Harbourfront",Intersection,Restaurant,Mexican Restaurant,Bank,Medical Center,Donut Shop,Breakfast Spot,Electronics Store,Rental Car Location,Diner


In [53]:
# Clustering 16 neighbourhoods into 4 clusters 

kclusters = 4
Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

In [54]:
# add clustering labels and top 10 venues to the Downtown bourough / meighbourhood dataframe 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Downtown_merged = Downtown_df
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# Sort by Cluster Labels 
Downtown_merged.sort_values(by=['Cluster Labels'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0.0,Intersection,Restaurant,Mexican Restaurant,Bank,Medical Center,Donut Shop,Breakfast Spot,Electronics Store,Rental Car Location,Diner
1,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0.0,General Entertainment,Skating Rink,Farm,Café,College Stadium,Baseball Field,Dessert Shop,Furniture / Home Store,French Restaurant,Food & Drink Shop
2,Downtown Toronto,St. James Town,43.799525,-79.318389,0.0,Pharmacy,Breakfast Spot,Fast Food Restaurant,Electronics Store,Coffee Shop,Pizza Place,Chinese Restaurant,Burger Joint,Sandwich Place,Furniture / Home Store
4,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0.0,Pizza Place,Supermarket,Coffee Shop,Pharmacy,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,College Stadium
5,Downtown Toronto,Christie,43.753259,-79.329656,0.0,Park,Pool,Food & Drink Shop,Yoga Studio,Electronics Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant
7,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389,0.0,Park,Beer Store,Skating Rink,Curling Ice,Falafel Restaurant,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store
8,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572,0.0,Park,Fast Food Restaurant,Pub,Gym,Ice Cream Shop,Italian Restaurant,Liquor Store,Movie Theater,Pet Store,Pizza Place
11,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.651571,-79.484450,0.0,Café,Pub,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,French Restaurant,Diner,Vegetarian / Vegan Restaurant,Gourmet Shop
12,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.662744,-79.321558,0.0,Garden,Skate Park,Comic Shop,Park,Light Rail Station,Pizza Place,Butcher,Burrito Place,Brewery,Restaurant
13,Downtown Toronto,Rosedale,43.636258,-79.498509,1.0,Baseball Field,Yoga Studio,Farmers Market,Diner,Discount Store,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant


In [55]:
#Dropping Berczy Park for visualization 
Downtown_merged = Downtown_merged.drop([3])
Downtown_merged.sort_values(by=['Cluster Labels'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0.0,Intersection,Restaurant,Mexican Restaurant,Bank,Medical Center,Donut Shop,Breakfast Spot,Electronics Store,Rental Car Location,Diner
1,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0.0,General Entertainment,Skating Rink,Farm,Café,College Stadium,Baseball Field,Dessert Shop,Furniture / Home Store,French Restaurant,Food & Drink Shop
2,Downtown Toronto,St. James Town,43.799525,-79.318389,0.0,Pharmacy,Breakfast Spot,Fast Food Restaurant,Electronics Store,Coffee Shop,Pizza Place,Chinese Restaurant,Burger Joint,Sandwich Place,Furniture / Home Store
4,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0.0,Pizza Place,Supermarket,Coffee Shop,Pharmacy,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,College Stadium
5,Downtown Toronto,Christie,43.753259,-79.329656,0.0,Park,Pool,Food & Drink Shop,Yoga Studio,Electronics Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant
7,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389,0.0,Park,Beer Store,Skating Rink,Curling Ice,Falafel Restaurant,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store
8,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572,0.0,Park,Fast Food Restaurant,Pub,Gym,Ice Cream Shop,Italian Restaurant,Liquor Store,Movie Theater,Pet Store,Pizza Place
11,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.651571,-79.484450,0.0,Café,Pub,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,French Restaurant,Diner,Vegetarian / Vegan Restaurant,Gourmet Shop
12,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.662744,-79.321558,0.0,Garden,Skate Park,Comic Shop,Park,Light Rail Station,Pizza Place,Butcher,Burrito Place,Brewery,Restaurant
13,Downtown Toronto,Rosedale,43.636258,-79.498509,1.0,Baseball Field,Yoga Studio,Farmers Market,Diner,Discount Store,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant


In [56]:
# visualizing clusters 

Downtown_merged['Cluster Labels'] = Downtown_merged['Cluster Labels'].apply(np.int64)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters